In [42]:
import pymongo
import mysql.connector
from mysql.connector.errors import IntegrityError
import re
from tqdm.notebook import tqdm
from reality_importer import BaseImporter
from datetime import datetime
from datetime import timedelta

In [43]:
mysql_connection = mysql.connector.connect(
    host='localhost', 
    user='ralfeus', 
    database='reality', 
    auth_plugin='auth_socket', 
    unix_socket='/var/run/mysqld/mysqld.sock'
)
mycursor = mysql_connection.cursor()

In [44]:
mongo_client = pymongo.MongoClient()
db = mongo_client['reality']

In [45]:
start_date = datetime(2020, 5, 10)
end_date = datetime(2020, 5, 10)
start_date_filter = {'timeAdded': {'$gte': start_date}} if start_date is not None else {}
end_date_filter = {'timeAdded': {'$lt': end_date + timedelta(days=1)}} if end_date is not None else {}

In [46]:
def add_product(entry, commit=False):
    mycursor.execute("INSERT INTO `product` (id, vendor, layout, floor_area, price," \
                     "latitude, longitude, closest_public_transport_stop_name," \
                     "closest_public_transport_stop_distance, offer_type, date_added) " \
                     "VALUES(%(id)s, %(vendor)s, %(layout)s, %(floor_area)s, %(price)s, " \
                     "%(latitude)s, %(longitude)s, %(closest_public_transport_stop_name)s," \
                     "%(closest_public_transport_stop_distance)s, %(offer_type)s, %(date_added)s)",
                    entry)
    if commit:
        mysql_connection.commit()
def commit():
    mysql_connection.commit()
def rollback():
    mysql_connection.rollback()

## SReality Sell (Prague)

- categories:
  - per_page': 999, # Amount of items per page
  - category_main_cb' - property type
      - 1 - Apartments
      - 2 - Houses
      - 3 - Land
      - 4 - Commercial
      - 5 - Other
  - category_type_cb' - offer type
      - 1 - Selling
      - 2 - Rent
  - category_sub_cb - layout
      - 2: '1+kk',
      - 3: '1+1',
      - 4: '2+kk',
      - 5: '2+1',
      - 6: '3+kk',
      - 7: '3+1',
      - 8: '4+kk',
      - 9: '4+1',
      - 10: '5+kk',
      - 11: '5+1',
      - 12: '6-a-vice'
  - 'locality_region_id': 10 # Praha

In [47]:
source_col = db['sreality']
layout = {
    2: '1+kk',
    3: '1+1',
    4: '2+kk',
    5: '2+1',
    6: '3+kk',
    7: '3+1',
    8: '4+kk',
    9: '4+1',
    10: '5+kk',
    11: '5+1',
    12: '6-a-vice'
}
filter = {'$and': [start_date_filter, end_date_filter]}
cursor = source_col.find(filter, 
                         {
                             'hash_id':1, 
                             'timeAdded':1, 
                             'name':1, 
                             'gps.lat':1,
                             'gps.lon':1, 
                             'price':1,
                             'seo':1
                         })
total = source_col.count_documents(filter)
for item in tqdm(cursor, total=total):
    closest_public_transport_stop = BaseImporter.getClosestStop(item['gps']['lat'], item['gps']['lon'])
    entry = {
        'id': str(item['hash_id']),
        'date_added': item['timeAdded'].date(),
        'vendor': 'sreality', 
        'name': item['name'],
        'latitude': item['gps']['lat'],
        'longitude': item['gps']['lon'],
        'price': item['price'],
        'layout': layout.get(item['seo']['category_sub_cb']),
        'floor_area': re.search('\s(\d+)\s', item['name']).groups()[0],
        'offer_type': 2 ** (item['seo']['category_type_cb'] - 1),
        'property_type': 2 ** (item['seo']['category_main_cb'] - 1),
        'closest_public_transport_stop_name': closest_public_transport_stop['name'],
        'closest_public_transport_stop_distance': closest_public_transport_stop['distance']
    }
    try:
        add_product(entry)
    except IntegrityError:
        pass
    except:
        rollback()
        raise
commit()

## SReality Rent

In [48]:
source_col = db['sreality_rent']
layout = {
    2: '1+kk',
    3: '1+1',
    4: '2+kk',
    5: '2+1',
    6: '3+kk',
    7: '3+1',
    8: '4+kk',
    9: '4+1',
    10: '5+kk',
    11: '5+1',
    12: '6-a-vice',
    47: 'pokoj'
}
filter = {'$and': [start_date_filter, end_date_filter]}
cursor = source_col.find(filter, 
                         {
                             'hash_id':1, 
                             'timeAdded':1, 
                             'name':1, 
                             'gps.lat':1,
                             'gps.lon':1, 
                             'price':1,
                             'seo':1
                         })
total = source_col.count_documents(filter)
for item in tqdm(cursor, total=total):
    closest_public_transport_stop = BaseImporter.getClosestStop(item['gps']['lat'], item['gps']['lon'])
    entry = {
        'id': str(item['hash_id']),
        'date_added': item['timeAdded'].date(),
        'vendor': 'sreality', 
        'name': item['name'],
        'latitude': item['gps']['lat'],
        'longitude': item['gps']['lon'],
        'price': item['price'],
        'layout': layout.get(item['seo']['category_sub_cb']),
        'floor_area': re.search('\s(\d+)\D', item['name']).groups()[0],
        'offer_type': 2 ** (item['seo']['category_type_cb'] - 1),
        'property_type': 2 ** (item['seo']['category_main_cb'] - 1),
        'closest_public_transport_stop_name': closest_public_transport_stop['name'],
        'closest_public_transport_stop_distance': closest_public_transport_stop['distance']
    }
    try:
        add_product(entry)
    except IntegrityError:
#         print(f'DUP: {entry["id"]}-{entry["date_added"]}')
        pass
    except:
        rollback()
        raise
commit()

## SReality All

## Bez realitky

In [37]:
source_col = db['bezrealitky']
offer_type = {
    'spolubydleni': 0,
    'prodej': 1,
    'pronajem': 2
}
property_type = {
    'byt': 1,
    'dum': 2,
    'pozemek': 3,
    'kancelar': 4,
    'garaz': 5,
    'nebytovy-prostor': 5,
    'rekreacni-objekt': 5
}
filter = {'$and': [start_date_filter, end_date_filter]}
cursor = source_col.find(filter, 
                         {
                             'id':1, 
                             'timeAdded':1, 
                             'advertEstateOffer':1
                         })
total = source_col.count_documents(filter)
for item in tqdm(cursor, total=total):
    gps = eval(item['advertEstateOffer'][0]['gps'])
    lat, lon = float(gps['lat']), float(gps['lng'])
    if (BaseImporter.prague_boundaries['south'] <= lat <= BaseImporter.prague_boundaries['north'] and
        BaseImporter.prague_boundaries['west'] <= lon <= BaseImporter.prague_boundaries['east'] and
        item['advertEstateOffer'][0]['keyEstateType'] == 'byt'):
        closest_public_transport_stop = BaseImporter.getClosestStop(lat, lon)
        entry = {
            'id': str(item['id']),
            'date_added': item['timeAdded'].date(),
            'vendor': 'Bez realitky', 
            'name': '',
            'latitude': lat,
            'longitude': lon,
            'price': str(item['advertEstateOffer'][0]['price']),
            'layout': item['advertEstateOffer'][0]['keyDisposition'].replace('-', '+'),
            'floor_area': item['advertEstateOffer'][0]['surface'],
            'offer_type': offer_type[item['advertEstateOffer'][0]['keyOfferType']],
            'property_type': 2 ** (property_type[item['advertEstateOffer'][0]['keyEstateType']] - 1),
            'closest_public_transport_stop_name': closest_public_transport_stop['name'],
            'closest_public_transport_stop_distance': closest_public_transport_stop['distance']
        }
        try:
            add_product(entry)
        except IntegrityError:
            pass
        except:
            mysql_connection.rollback()
            raise
commit()

## Central Group

In [49]:
source_col = db['centralGroup']
filter = {'$and': [start_date_filter, end_date_filter]}
cursor = source_col.find(filter, 
                         {
                             'catalogNumber':1, 
                             'timeAdded':1, 
                             'totalPriceWithVAT':1,
                             'layoutLabel':1,
                             'totalFloorArea':1
                         })
total = source_col.count_documents(filter)
for item in tqdm(cursor, total=total):
    entry = {
        'id': item['catalogNumber'],
        'date_added': item['timeAdded'].date(),
        'vendor': 'Central Group', 
        'name': '',
        'latitude': 0,
        'longitude': 0,
        'price': item['totalPriceWithVAT'],
        'layout': item['layoutLabel'],
        'floor_area': item['totalFloorArea'],
        'offer_type': 1,
        'property_type': 1,
        'closest_public_transport_stop_name': '',
        'closest_public_transport_stop_distance': 0
    }
    try:
        add_product(entry)
    except IntegrityError:
        pass
    except:
        rollback()
        raise
commit()

## Ekospol

In [39]:
source_col = db['ekospol']
filter = {'$and': [start_date_filter, end_date_filter]}
cursor = source_col.find(filter, 
                         {
                             'identification':1, 
                             'timeAdded':1, 
                             'priceWithVAT':1,
                             'layout':1,
                             'totalFloorArea':1
                         })
total = source_col.count_documents(filter)
for item in tqdm(cursor, total=total):
    entry = {
        'id': item['identification'],
        'date_added': item['timeAdded'].date(),
        'vendor': 'Ekospol', 
        'name': '',
        'latitude': 0,
        'longitude': 0,
        'price': item['priceWithVAT'],
        'layout': item['layout'],
        'floor_area': item['totalFloorArea'],
        'offer_type': 1,
        'property_type': 1,
        'closest_public_transport_stop_name': '',
        'closest_public_transport_stop_distance': 0
    }
    try:
        add_product(entry)
    except IntegrityError:
        pass
    except:
        rollback()
        raise
commit()

## Finep

In [40]:
source_col = db['finep']
filter = {'$and': [start_date_filter, end_date_filter]}
cursor = source_col.find(filter, 
                         {
                             'identification':1, 
                             'timeAdded':1, 
                             'priceWithVAT':1,
                             'layout':1,
                             'totalFloorArea':1
                         })
total = source_col.count_documents(filter)
for item in tqdm(cursor, total=total):
    entry = {
        'id': item['identification'],
        'date_added': item['timeAdded'].date(),
        'vendor': 'Finep', 
        'name': '',
        'latitude': 0,
        'longitude': 0,
        'price': item['priceWithVAT'],
        'layout': item['layout'],
        'floor_area': item['totalFloorArea'],
        'offer_type': 1,
        'property_type': 1,
        'closest_public_transport_stop_name': '',
        'closest_public_transport_stop_distance': 0
    }
    try:
        add_product(entry)
    except IntegrityError:
        pass
    except:
        rollback()
        raise
commit()

## Skanska

In [41]:
source_col = db['skanska']
filter = {'$and': [start_date_filter, end_date_filter]}
cursor = source_col.find(filter, 
                         {
                             'identification':1, 
                             'timeAdded':1, 
                             'priceWithVAT':1,
                             'layout':1,
                             'totalFloorArea':1
                         })
total = source_col.count_documents(filter)
for item in tqdm(cursor, total=total):
    entry = {
        'id': item['identification'],
        'date_added': item['timeAdded'].date(),
        'vendor': 'Skanska', 
        'name': '',
        'latitude': 0,
        'longitude': 0,
        'price': item['priceWithVAT'],
        'layout': item['layout'],
        'floor_area': item['totalFloorArea'],
        'offer_type': 1,
        'property_type': 1,
        'closest_public_transport_stop_name': '',
        'closest_public_transport_stop_distance': 0
    }
    try:
        add_product(entry)
    except IntegrityError:
        pass
    except:
        rollback()
        raise
commit()